In [1]:
# Import Libraries
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
# Send Get request data
WIKI_URL = "https://en.wikipedia.org/w/api.php?formatversion=2&action=parse&page=List_of_most_visited_museums&prop=text&format=json"
response = requests.get(url=WIKI_URL)
wiki_data = json.loads(response.text)
wiki_text = wiki_data["parse"]["text"]



In [3]:
# Parse text
soup = bs(wiki_text, "lxml")
table = soup.find('table')
list_of_rows = []

for row in table.findAll('tr'):  # find row
    list_of_cells = []
    for cell in row.findAll('td'):  # each cell
        # if country in it get country
        if cell.find('a') and cell.find('a').get('title'):
            if cell.find('a').get('class') and 'new' in cell.find('a').get('class'):
                text = cell.text  # A special case
                list_of_cells.append(text.replace(r'\xa0[zh', '').strip())
                continue
            text = cell.find('a').get('title')  # get country
            if cell.find('span'):
                city = cell.text
                list_of_cells.append(city.strip())
        else:  # else get text only
            text = cell.text
        list_of_cells.append(text.strip())
    if list_of_cells:
        list_of_rows.append(list_of_cells)



In [4]:
# Convert into dataframe
df = pd.DataFrame(list_of_rows, columns=['Museum_Name',
                                         'City',
                                         'Country',
                                         'Number_Of_Visitors',
                                         'Year_Reported'])
df['Number_Of_Visitors'] = df['Number_Of_Visitors'].map(lambda num: int(str(num).replace(',', '')))
df['Year_Reported'] = df['Year_Reported'].map(lambda num: int(str(num)[:4]))

In [5]:
# Import database
from db import Database, City

In [6]:
# Create function to get city population
CITY_DATABASE_URI = 'sqlite:///city_info.sqlite'  # db path

city_db = Database(db_uri=CITY_DATABASE_URI)

def get_city_population(city: str, country: str):
    population_results = city_db.session.query(City).filter(City.city == city).first()
    if population_results:
        return population_results.population
    else:
        return None

In [7]:
# Get population into dataframe
city_population_lst = []
for iid, row in df.iterrows():
    city_population = get_city_population(row['City'], row['Country'])
    city_population_lst.append(int(city_population) if city_population else None)



In [8]:
# Save in db
MUSEUM_DATABASE_URI = 'sqlite:///museum.sqlite'  # db path

df['Population_Of_City'] = city_population_lst
db = Database(db_uri=MUSEUM_DATABASE_URI)
db.create_tables()
df.to_sql('museum', index=False, con=db.engine, if_exists='replace')

In [ ]:
#访问人数
visited_people = df['Number_Of_Visitors']
#当地人口
population = df['Population_Of_City']
